In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
det_cats = {
    'pz14' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv'),
    'pz20' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz20_thr0.1_step8.csv'),
    'pz25' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz25_thr0.1_step8.csv'),
    'pz40' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz40_thr0.1_step8.csv'),
    'pz_act10' : pd.read_csv(
    '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv'),
    'pz_act14' : pd.read_csv(
    '/home/rt2122/Data/detected_cats/full_pz_act14_thr0.1_step8.csv'),
    'pz_act20' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv'),
    'pz_act25' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act25_thr0.1_step8.csv')
}

In [3]:
psz2 = None
with fits.open('/home/rt2122/Data/original_catalogs/psz2.fits') as hdul:
    data = hdul[1].data
    psz2 = Table(data).to_pandas()
mcxc = None
with fits.open('/home/rt2122/Data/original_catalogs/mcxc.fits') as hdul:
    data = hdul[1].data
    mcxc = Table(data).to_pandas()
rm = None
with fits.open('/home/rt2122/Data/original_catalogs/redmapper.fits.gz') as hdul:
    data = Table(hdul[1].data)
    names = [name for name in data.colnames if len(data[name].shape) <= 1]
    rm = data[names].to_pandas()
act = None
with fits.open('/home/rt2122/Data/original_catalogs/act.fits') as hdul:
    data = hdul[1].data
    act = Table(data).to_pandas()

In [4]:
true_cats = {'psz2' : psz2, 'mcxc' : mcxc, 'rm' : rm, 'act' : act}

In [5]:
true_cats['mcxc'].rename({'RAdeg' : 'RA', 'DEdeg' : 'DEC'}, axis='columns', inplace=True)
true_cats['act'].rename({'RADeg' : 'RA', 'decDeg' : 'DEC'}, axis='columns', inplace=True)

In [6]:
match_dist = 5 / 60

In [11]:
true_cats['planck_z'] = pd.read_csv('/home/rt2122/Data/clusters/planck_z.csv')

In [15]:
true_cats['planck_no_z'] = pd.read_csv('/home/rt2122/Data/clusters/planck_no_z.csv')
true_cats['mcxcwp'] = pd.read_csv('/home/rt2122/Data/clusters/mcxcwp.csv')
true_cats['actwp'] = pd.read_csv('/home/rt2122/Data/clusters/act.csv')

In [16]:
comp_df = []
recall_df = []
for det_name in det_cats:
    line = {}
    line_r = {}
    
    sc = SkyCoord(ra=np.array(det_cats[det_name]['RA'])*u.degree, 
                  dec=np.array(det_cats[det_name]['DEC'])*u.degree, frame='icrs')
    
    for tr_name in true_cats: 
        tr_sc = SkyCoord(ra=np.array(true_cats[tr_name]['RA'])*u.degree, 
                      dec=np.array(true_cats[tr_name]['DEC'])*u.degree, frame='icrs')
        idx, d2d, _ = sc.match_to_catalog_sky(tr_sc)
        matched = d2d.degree <= match_dist
        line[tr_name] = np.count_nonzero(det_cats[det_name].iloc[matched]['status'] != 'fn')
        line[tr_name+'_err'], line[tr_name+'_std'] = calc_error(det_cats[det_name], true_cats[tr_name])
        
        line_r[tr_name] = line[tr_name] / len(true_cats[tr_name])
    line['fp'] = np.count_nonzero(det_cats[det_name]['status'] == 'fp')
    line_r['fp'] = line['fp']
    comp_df.append(pd.DataFrame(line, index=[det_name]))
    recall_df.append(pd.DataFrame(line_r, index=[det_name]))

line = {}
for tr_name in true_cats:
    line[tr_name] = len(true_cats[tr_name])
    line[tr_name+'_err'] = 0
line['fp'] = 0
comp_df.append(pd.DataFrame(line, index=['all']))
comp_df = pd.concat(comp_df)
recall_df = pd.concat(recall_df)

In [19]:
comp_df[['psz2', 'psz2_err', 'psz2_std', 'mcxc', 'mcxc_err', 'mcxc_std', 'rm', 'rm_err', 'act', 'act_err', 'act_std']]

,psz2,psz2_err,psz2_std,mcxc,mcxc_err,mcxc_std,rm,rm_err,act,act_err,act_std
pz14,1491,21.70,1.167769,725,24.15,0.795630,1242,401.30,849,67.55,1.746387
pz20,1528,24.50,1.050063,740,26.20,0.961359,1306,413.55,875,74.20,2.262509
pz25,1525,25.35,0.963205,747,27.70,0.852550,1374,447.55,888,77.45,2.343496
pz40,1506,21.65,1.117504,739,24.70,0.909309,1279,422.90,871,73.25,1.385783
pz_act10,1361,17.00,0.867543,659,18.60,0.966273,1029,292.70,926,47.75,1.725772
pz_act14,1363,18.00,1.063757,670,21.20,1.087247,1211,335.30,1282,56.45,1.302780
pz_act20,1217,14.95,0.856784,599,15.45,0.662829,1182,270.60,1866,51.90,2.062063
pz_act25,1260,18.65,1.024117,610,17.00,1.112134,1271,291.80,2152,55.55,1.563523
all,1653,0.00,NaN,1743,0.00,NaN,26111,0.00,4195,0.00,NaN


In [18]:
recall_df

,psz2,mcxc,rm,act,planck_z,planck_no_z,mcxcwp,actwp,fp
pz14,0.901996,0.415950,0.047566,0.202384,0.971664,0.765653,0.178541,0.122043,15828
pz20,0.924380,0.424555,0.050017,0.208582,0.990859,0.794275,0.185247,0.128226,23104
pz25,0.922565,0.428571,0.052622,0.211681,0.990859,0.788909,0.189438,0.131989,20611
pz40,0.911071,0.423982,0.048983,0.207628,0.978976,0.778175,0.187762,0.128226,17306
pz_act10,0.823351,0.378084,0.039409,0.220739,0.911335,0.651163,0.135792,0.152151,16316
pz_act14,0.824561,0.384395,0.046379,0.305602,0.916819,0.644007,0.144174,0.245699,16484
pz_act20,0.736237,0.343660,0.045268,0.444815,0.865631,0.483005,0.098910,0.409140,9398
pz_act25,0.762250,0.349971,0.048677,0.512992,0.876600,0.538462,0.107293,0.483871,15275


In [7]:
def calc_error(det_cat, true_cat, shift=15/60, match_dist=5/60, n_try=20, seed=0):
    import numpy as np
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    error = []
    np.random.seed(seed)
    for i in range(n_try):
        det_sc = SkyCoord(ra=np.array(det_cat['RA']) * u.degree, 
                          dec=np.array(det_cat['DEC']) * u.degree, frame='icrs')
        angles = np.random.randint(0, 360, len(det_cat))
        det_sc = det_sc.directional_offset_by(angles*u.degree, shift)

        true_sc = SkyCoord(ra=np.array(true_cat['RA']) * u.degree, 
                           dec=np.array(true_cat['DEC']) * u.degree, frame='icrs')
        _, d2d, _ = det_sc.match_to_catalog_sky(true_sc)
        c_error = np.count_nonzero(d2d.degree < match_dist)
        error.append(c_error)
    error = np.array(error)
    return error.mean(), error.std() / np.sqrt(n_try - 1)